In [9]:
import pandas as pd

# Load dataset
df = pd.read_csv("dataset.csv")

# Only these features
features = ["tempo", "loudness", "danceability"]

# Quartiles
quartiles = df[features].quantile([0.25, 0.5, 0.75])
print(quartiles)


          tempo  loudness  danceability
0.25   99.21875   -10.013         0.456
0.50  122.01700    -7.004         0.580
0.75  140.07100    -5.003         0.695


In [10]:
def mood_mapper(tempo, loudness, danceability, q):
    # Loudness category
    if loudness >= q["loudness"].loc[0.75]:
        loud_cat = "high"
    elif loudness <= q["loudness"].loc[0.25]:
        loud_cat = "low"
    else:
        loud_cat = "mid"
    
    # Tempo category
    if tempo >= q["tempo"].loc[0.75]:
        tempo_cat = "fast"
    elif tempo <= q["tempo"].loc[0.25]:
        tempo_cat = "slow"
    else:
        tempo_cat = "mid"
    
    # Danceability category
    if danceability >= q["danceability"].loc[0.75]:
        dance_cat = "high"
    elif danceability <= q["danceability"].loc[0.25]:
        dance_cat = "low"
    else:
        dance_cat = "mid"

    # Mood rules (relaxed, not over-constrained)
    if tempo_cat == "fast" and dance_cat == "high" and loud_cat == "high":
        return "Happy / Energetic (High Beat Party)"
    
    elif tempo_cat in ["mid", "fast"] and dance_cat in ["mid", "high"] and loud_cat in ["mid", "high"]:
        return "Happy / Energetic (Pleasant)"
    
    elif tempo_cat == "mid" and dance_cat == "mid":
        return "Uplifting / Positive"
    
    elif tempo_cat == "slow" and loud_cat == "low":
        return "Calm / Peaceful"
    
    elif tempo_cat == "slow" and dance_cat == "low":
        return "Relaxation / Safety"
    
    elif tempo_cat == "mid" and dance_cat in ["mid", "low"]:
        return "Balanced / Neutral"
    
    else:
        # Instead of dumping, force it into closest vibe
        if tempo_cat == "fast":
            return "High Energy / Excited"
        elif tempo_cat == "slow":
            return "Calm / Peaceful"
        else:
            return "Balanced / Neutral"


In [11]:
df["MoodCategory"] = df.apply(
    lambda row: mood_mapper(row["tempo"], row["loudness"], row["danceability"], quartiles),
    axis=1
)

# Save
df.to_csv("dataset_with_moods_tld.csv", index=False)

# Distribution
print(df["MoodCategory"].value_counts())


MoodCategory
Happy / Energetic (Pleasant)           51442
Calm / Peaceful                        24577
Balanced / Neutral                     13245
High Energy / Excited                  13221
Uplifting / Positive                    6436
Relaxation / Safety                     3923
Happy / Energetic (High Beat Party)     1156
Name: count, dtype: int64


In [21]:
merge_map={
    "High Energy / Excited": "Happy / Energetic (High Beat Party)",
    "Relaxation / Safety":"Calm / Peaceful",
}
df["MoodCategory"]=df["MoodCategory"].replace(merge_map)

In [22]:
print(df["MoodCategory"].value_counts())

MoodCategory
Happy / Energetic (Pleasant)           51442
Calm / Peaceful                        28500
Happy / Energetic (High Beat Party)    14377
Balanced / Neutral                     13245
Uplifting / Positive                    6436
Name: count, dtype: int64


In [23]:
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,MoodCategory
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,Calm / Peaceful
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,Calm / Peaceful
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,Calm / Peaceful
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,Happy / Energetic (High Beat Party)
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,Happy / Energetic (Pleasant)
